## Retrieveing cafes and peaks in Salzburg

In [284]:
# Import libraries
import folium
from folium import DivIcon, Tooltip
from folium.plugins import MarkerCluster
import requests

In [285]:
# Install missing library
!pip install folium

In [295]:
# Define Salzburg city center
map_salzburg = folium.Map(
    location=[47.80, 13.03],
    zoom_start=13
)

In [297]:
# 
folium.TileLayer(
    tiles='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png',
    attr='© OSM, © CARTO',
    name='Clean Base'
).add_to(map_salzburg)


In [298]:
# Overpass API query for cafes in Salzburg
cafes_query = """
[out:json];
node["amenity"="cafe"]["outdoor_seating"="yes"](47.75,13.00,47.85,13.10);
out;
"""
cafes_url = "http://overpass-api.de/api/interpreter"
cafes_response = requests.post(cafes_url, data=cafes_query)
cafes_data = cafes_response.json()

In [299]:
# Add cafes to the map
# Create a MarkerCluster group
cafe_cluster = MarkerCluster().add_to(map_salzburg)

# Add cafe markers to the cluster
for element in cafes_data["elements"]:
    name = element["tags"].get("name", "Unnamed Cafe")
    lat = element["lat"]
    lon = element["lon"]

    folium.Marker(
        location=[lat, lon],
        popup=f"Cafe: <b>{name}</b>",
        icon=folium.Icon(color="orange", icon="coffee", prefix="fa"),
        tooltip=Tooltip(f"{name}")
    ).add_to(cafe_cluster)


In [300]:
# Return number of cafes with outdoor service retrieved
num_cafes = len(cafes_data.get("elements", []))
print(f"Number of cafes retrieved: {num_cafes}")

Number of cafes retrieved: 52


In [301]:
map_salzburg

In [302]:
# Overpass API query for mountain peaks
peaks_query = """
[out:json];
node["natural"="peak"](47.79,13.01,47.83,13.07);
out;
"""

peaks_response = requests.post(cafes_url, data=peaks_query)
peaks_data = peaks_response.json()

In [303]:
# Add peaks to the map
for element in peaks_data["elements"]:
    name = element["tags"].get("name", "Unnamed Peak")
    ele = element["tags"].get("ele", "N/A")
    lat = element["lat"]
    lon = element["lon"]

    triangle_icon_html = '<i class="fa fa-play" style="color:green; font-size:14px; transform: rotate(90deg);"></i>'

    folium.Marker(
        location=[lat, lon],
        icon=DivIcon(
            icon_size=(14, 14),
            icon_anchor=(7, 7),
            html=triangle_icon_html,
        ),
        tooltip=Tooltip(f"{name} ({ele} m)")
    ).add_to(map_salzburg)



In [304]:
map_salzburg

In [305]:
# Save to HTML file
map_salzburg.save("salzburg_cafes_and_peaks.html")


In [306]:
from folium import Element

legend_html = """
<div style="
    position: fixed; 
    bottom: 30px; left: 30px; width: 160px; 
    background-color: white;
    border:2px solid grey;
    z-index:9999;
    font-size:14px;
    padding: 10px;
    box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
    border-radius: 8px;
">
<b>Legend</b><br>
<i class="fa fa-coffee" style="color:orange"></i> Café<br>
<i class="fa fa-play" style="color:green; transform: rotate(90deg);"></i> Peak<br>
</div>
"""

map_salzburg.get_root().html.add_child(Element(legend_html))


In [307]:
map_salzburg